## Grupo MotoTrack

## integrantes:
    
    RM558317 - Cauã Sanches de Santana

    RM556511 - Angello Turano da Costa

    RM558576 - Leonardo Bianchi

## Solução: 

Sensores RFID (Radio Frequency Identification)
Os sensores RFID oferecem uma solução mais eficaz para localização dentro de ambientes fechados, como depósitos e pátios. Eles funcionam de maneira diferente do GPS, usando etiquetas RFID nas motos e leitores RFID posicionados no pátio para rastrear a posição das motos.

Como o RFID pode ser usado:
Precisão: RFID oferece alta precisão em ambientes fechados, com a capacidade de determinar a posição das motos com uma margem de erro muito menor, em torno de 1 a 3 metros, dependendo da configuração dos leitores.

Funcionamento: Cada moto teria um etiqueta RFID passiva ou ativa. As etiquetas passivas podem ser lidas por leitores de RFID instalados ao longo do depósito, enquanto as etiquetas ativas podem emitir sinais de forma mais autônoma. Quando uma moto se aproxima de um leitor, a posição da moto é registrada no sistema.

Leitores em Posições Estratégicas: Para garantir uma cobertura completa do pátio, leitores de RFID seriam posicionados estrategicamente ao longo das áreas do depósito. A cada vez que uma moto passa por um leitor, a localização da moto é registrada automaticamente.

Vantagens do RFID para a Localização das Motos:
Alta Precisão em Ambientes Fechados: RFID é a melhor opção quando se trata de rastrear a posição exata de objetos dentro de um espaço fechado, como um pátio de motos.

Escalabilidade: A instalação de leitores RFID no pátio pode ser feita de forma modular e escalável, conforme a necessidade de cobertura de área.

Baixo Custo e Manutenção: Sensores RFID são mais baratos e têm baixa necessidade de manutenção em comparação com tecnologias como GPS, especialmente para ambientes internos.

##  1. Geração do Dataset Simulado com Dados de Motos da Mottu
Este código simula um conjunto de dados fictício com 5.000 registros representando dispositivos Bluetooth acoplados a motos da Mottu. Cada entrada possui:

Um identificador único (ex: BT-00001)

Modelo da moto (ex: "Mottu Sport", "Mottu E", "Mottu Pop")

Condição da moto ("Nova", "Usada", "Manutenção")

Localização da moto em um dos cinco pátios da empresa

A distribuição dos dados segue pesos realistas e ajustados por tipo de moto, sugerindo que modelos mais populares estejam mais frequentemente em determinados pátios ou condições específicas. O resultado final é salvo como um arquivo CSV chamado motos_dataset.csv.

##

In [ ]:
import pandas as pd
import random

# Listas de valores possíveis
modelos = ["Mottu Sport", "Mottu E", "Mottu Pop"]
condicoes = ["Nova", "Usada", "Manutenção"]
patios = ["Unidade Belém", "Unidade Tatuapé", "Unidade Santo Amaro", "Unidade Mooca", "Unidade Lapa"]
algoritmos_visao = ["YOLOv5", "YOLOv8", "MobileNet-SSD"]

# Função para simular dados de visão computacional
def gerar_dados_visao():
    prob = random.random()
    
    if prob < 0.4:
        modelo = "Mottu Sport"
        condicao = random.choices(condicoes, weights=[0.7, 0.2, 0.1])[0]
        patio = random.choices(patios, weights=[0.1, 0.1, 0.6, 0.1, 0.1])[0]
    elif prob < 0.7:
        modelo = "Mottu E"
        condicao = random.choices(condicoes, weights=[0.2, 0.3, 0.5])[0]
        patio = random.choices(patios, weights=[0.05, 0.1, 0.05, 0.7, 0.1])[0]
    else:
        modelo = "Mottu Pop"
        condicao = random.choices(condicoes, weights=[0.1, 0.85, 0.05])[0]
        patio = random.choices(patios, weights=[0.3, 0.2, 0.1, 0.1, 0.3])[0]

    # Simular coordenadas X e Y dentro de um pátio de 100x100 metros
    x_coord = round(random.uniform(0, 100), 2)
    y_coord = round(random.uniform(0, 100), 2)

    # Simular algoritmo usado e frame detectado
    algoritmo = random.choice(algoritmos_visao)
    frame_id = random.randint(1, 1000)

    return modelo, condicao, patio, x_coord, y_coord, algoritmo, frame_id

# Criar dataset com 5000 registros simulados
n = 5000
data = []
for i in range(1, n + 1):
    modelo, condicao, patio, x, y, algoritmo, frame = gerar_dados_visao()
    data.append([f"CAM-{i:05d}", modelo, condicao, patio, x, y, algoritmo, frame])

# Criar DataFrame e salvar em CSV
df = pd.DataFrame(data, columns=["Identificador Visual", "Modelo", "Condição", "Pátio", "Coord_X", "Coord_Y", "Algoritmo_Visao", "Frame_ID"])
df.to_csv("motoTrack_dataset.csv", index=False)
print("CSV com dados simulados de visão computacional salvo como 'motoTrack_dataset.csv'")

## 2. Pré-processamento com LabelEncoder para Machine Learning
Este trecho de código é responsável pelo tratamento dos dados categóricos contidos no dataset gerado:

Aplica LabelEncoder para transformar os textos das colunas Modelo, Condição e Pátio em valores numéricos, permitindo que algoritmos de machine learning possam trabalhar com eles.

Define as variáveis independentes (X) com os dados da condição e do pátio da moto, e a variável dependente (y) como o modelo da moto.

Esse processo é fundamental para preparar os dados antes do treinamento do modelo de classificação.



In [ ]:
# Reimportar pandas e carregar o novo dataset de visão computacional
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Carregar dataset simulado de visão
df = pd.read_csv("motos_visao_dataset.csv")

# Instanciar os LabelEncoders
le_cond    = LabelEncoder()
le_patio   = LabelEncoder()
le_modelo  = LabelEncoder()
le_alg     = LabelEncoder()

# Ajustar e transformar cada coluna categórica
df['Cond_Enc']   = le_cond.fit_transform( df['Condição'] )
df['Patio_Enc']  = le_patio.fit_transform( df['Pátio'] )
df['Modelo_Enc'] = le_modelo.fit_transform( df['Modelo'] )
df['Alg_Enc']    = le_alg.fit_transform( df['Algoritmo_Visao'] )

# Selecionar features e target
# - Incluímos também Coord_X e Coord_Y para treinar um modelo que aprenda a posição a partir
#   das características da moto, pátio e algoritmo de visão.
X = df[['Cond_Enc', 'Patio_Enc', 'Alg_Enc', 'Coord_X', 'Coord_Y']]
y = df['Modelo_Enc']

# Exemplo: visualizar as primeiras linhas
print(X.head())
print(y[:5])


##  3. Treinamento do Modelo com XGBoost
Nesta etapa, os dados preparados são utilizados para treinar um modelo de classificação multi-classe com XGBoost, um dos algoritmos mais eficientes para esse tipo de tarefa:

Os dados são divididos em treino (70%) e teste (30%).

O modelo XGBClassifier é configurado com hiperparâmetros ajustados manualmente para melhor desempenho.

Após o treinamento, o modelo realiza previsões com base nos dados de teste.

A acurácia da previsão é calculada com accuracy_score, indicando o quão bem o modelo consegue identificar corretamente o modelo da moto com base no pátio e na condição.

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Instanciar o modelo com hiperparâmetros otimizados
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',
    max_depth=6,
    learning_rate=0.1,
    n_estimators=150,
    random_state=42
)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Calcular acurácia
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia no modelo com dados de visão computacional: {acc:.4f}")

## 4. Visualização com Matriz de Confusão 
Este código gera uma matriz de confusão visual, uma ferramenta essencial para avaliar o desempenho do modelo:

Compara os valores previstos com os reais, mostrando onde o modelo acertou ou errou.

A matriz é plotada com seaborn.heatmap() usando o esquema de cores preto e amarelo ("cividis"), que melhora a visualização e acessibilidade.

Inclui títulos, rótulos e anotações personalizadas, com foco em clareza e estética.

Essa visualização ajuda a entender quais modelos de motos estão sendo confundidos entre si pelo algoritmo.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
labels = le_modelo.classes_

# Plotar heatmap com cores preto e amarelo
plt.figure(figsize=(9, 7))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='cividis',  # tons de amarelo e preto
    linewidths=0.6,
    linecolor='black',
    xticklabels=labels,
    yticklabels=labels,
    annot_kws={"size": 14, "weight": "bold", "color": "black"}
)

# Títulos e rótulos
plt.title('Matriz de Confusão — Previsão de Modelos de Motos', fontsize=16, pad=20, color='black')
plt.xlabel('Modelo Previsto', fontsize=13, color='black')
plt.ylabel('Modelo Real', fontsize=13, color='black')
plt.xticks(fontsize=11, color='black')
plt.yticks(fontsize=11, color='black')
plt.tight_layout()
plt.show()